<a href="https://colab.research.google.com/github/ShadmanRohan/haiku-jax-notes/blob/main/Simple_MLP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Preliminaries**

In [3]:
!pip install dm-haiku
import jax
import haiku as hk
from sklearn import datasets
from jax import numpy as jnp
from sklearn.model_selection import train_test_split
print("Haiku Version :{}".format(hk.__version__))
print("JAX Version : {}".format(jax.__version__))

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Haiku Version :0.0.8
JAX Version : 0.3.23


**Dataset**

In [4]:
X, Y = datasets.load_boston(return_X_y=True)
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, train_size=0.8, random_state=123)
X_train, X_test, Y_train, Y_test = jnp.array(X_train, dtype=jnp.float32),\
                                   jnp.array(X_test, dtype=jnp.float32),\
                                   jnp.array(Y_train, dtype=jnp.float32),\
                                   jnp.array(Y_test, dtype=jnp.float32),\

samples, features = X_train.shape
X_train.shape, X_test.shape, Y_train.shape, Y_test.shape


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np


        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.

((404, 13), (102, 13), (404,), (102,))

In [5]:
samples, features

(404, 13)

In [6]:
mean = X_train.mean(axis=0)
std = X_train.std(axis=0)

X_train = (X_train - mean) / std
X_test = (X_test - mean) / std

In [7]:
def FeedForward(x):
    mlp = hk.nets.MLP(output_sizes=[5,10,15,1])
    return mlp(x)

model = hk.transform(FeedForward)


In [8]:
rng = jax.random.PRNGKey(42)
params = model.init(rng, X_train[:5])
print("Weights Type : {}\n".format(type(params)))

for layer_name, weights in params.items():
    print(layer_name)
    print("Weights : {}, Biases : {}\n".format(params[layer_name]["w"].shape,params[layer_name]["b"].shape))

Weights Type : <class 'dict'>

mlp/~/linear_0
Weights : (13, 5), Biases : (5,)

mlp/~/linear_1
Weights : (5, 10), Biases : (10,)

mlp/~/linear_2
Weights : (10, 15), Biases : (15,)

mlp/~/linear_3
Weights : (15, 1), Biases : (1,)



In [9]:
preds = model.apply(params, rng, X_train)
preds[:5]

DeviceArray([[-0.7874111 ],
             [-0.2776872 ],
             [-0.01174069],
             [-0.01407542],
             [-0.38728935]], dtype=float32)

In [10]:
def MeanSquaredErrorLoss(weights, input_data, actual):
    preds = model.apply(weights, rng, input_data)
    preds = preds.squeeze()
    return jnp.power(actual - preds, 2).mean()

In [11]:
def UpdateWeights(weights,gradients):
    return weights - learning_rate * gradients

In [12]:
from jax import value_and_grad

rng = jax.random.PRNGKey(42) ## Reproducibility ## Initializes model with same weights each time.

params = model.init(rng, X_train[:5])
epochs = 1000
learning_rate = jnp.array(0.001)

for i in range(1, epochs+1):
    loss, param_grads = value_and_grad(MeanSquaredErrorLoss)(params, X_train, Y_train)
    params = jax.tree_map(UpdateWeights, params, param_grads)

    if i%100 == 0:
        print("MSE : {:.2f}".format(loss))


MSE : 17.04
MSE : 12.32
MSE : 10.59
MSE : 9.66
MSE : 9.09
MSE : 8.76
MSE : 8.55
MSE : 8.38
MSE : 8.23
MSE : 8.13


In [13]:
train_preds = model.apply(params, rng, X_train)

train_preds[:5]


DeviceArray([[48.474415],
             [11.665481],
             [21.027842],
             [26.18423 ],
             [15.279279]], dtype=float32)

In [14]:
test_preds = model.apply(params, rng, X_test)
test_preds[:5]

DeviceArray([[20.905386],
             [25.025269],
             [44.169964],
             [21.290577],
             [29.036097]], dtype=float32)

In [15]:
print("Test  MSE Score : {:.2f}".format(MeanSquaredErrorLoss(params, X_test, Y_test)))
print("Train MSE Score : {:.2f}".format(MeanSquaredErrorLoss(params, X_train, Y_train)))

Test  MSE Score : 18.38
Train MSE Score : 8.13
